# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [3]:
sum(list(square_root_generator(limit)))

8.382332347441762

In [5]:
list(square_root_generator(13))

[1.0,
 1.4142135623730951,
 1.7320508075688772,
 2.0,
 2.23606797749979,
 2.449489742783178,
 2.6457513110645907,
 2.8284271247461903,
 3.0,
 3.1622776601683795,
 3.3166247903554,
 3.4641016151377544,
 3.605551275463989]

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data




In [13]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

print(f"\n\n\n")

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}




{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [ ]:
#Install the dependencies
%%capture
%pip install dlt[duckdb]

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

In [21]:
# %pip install dlt[duckdb]
import dlt

# define the connection to load to.
# We now use duckdb (dlt is database agnostic)
generators_pipeline = dlt.pipeline(destination="duckdb", dataset_name="generators")


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(
    people_1(), table_name="people", write_disposition="replace"
)

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)



Pipeline dlt_ipykernel_launcher load step completed in 0.30 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\Nguyen\OneDrive\Documents\GitHub\data-engineering-zoomcamp\cohorts\2024\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1719119179.8666902 is LOADED and contains no failed jobs


In [22]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people_info = conn.sql("SELECT * FROM people").df()
display(people_info)


# calculate the sum of ages of all people
print(f"\n\n\n the sum of ages of all people: {people_info["age"].sum()}")

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_new          │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1719119179.8666902,ySvEZ4ihJTIBng,None
1,2,Person_2,27,City_A,1719119179.8666902,MScEKX4gWZdd+g,None
2,3,Person_3,28,City_A,1719119179.8666902,+8uhShWSrngu8w,None
3,4,Person_4,29,City_A,1719119179.8666902,R+jyTuJXDnf9jw,None
4,5,Person_5,30,City_A,1719119179.8666902,HsrtfovA+aF92g,None





 the sum of ages of all people: 140


2. Append the second generator to the same table as the first.

In [23]:
# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(
    people_2(),
    table_name="people",
    write_disposition="append",
)

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.28 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\Nguyen\OneDrive\Documents\GitHub\data-engineering-zoomcamp\cohorts\2024\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1719119186.90434 is LOADED and contains no failed jobs


3. **After correctly appending the data, calculate the sum of all ages of people.**

In [24]:
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people_info = conn.sql("SELECT * FROM people").df()
display(people_info)


# calculate the sum of ages of all people
print(f"\n\n\n the sum of ages of all people: {people_info["age"].sum()}")

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_new          │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1719119179.8666902,ySvEZ4ihJTIBng,None
1,2,Person_2,27,City_A,1719119179.8666902,MScEKX4gWZdd+g,None
2,3,Person_3,28,City_A,1719119179.8666902,+8uhShWSrngu8w,None
3,4,Person_4,29,City_A,1719119179.8666902,R+jyTuJXDnf9jw,None
4,5,Person_5,30,City_A,1719119179.8666902,HsrtfovA+aF92g,None
5,3,Person_3,33,City_B,1719119186.90434,o383UEgWUY/gaA,Job_3
6,4,Person_4,34,City_B,1719119186.90434,eGABtPLIvJLJtw,Job_4
7,5,Person_5,35,City_B,1719119186.90434,Ku42mPWjAwjLxA,Job_5
8,6,Person_6,36,City_B,1719119186.90434,SbLBENfPweXylw,Job_6
9,7,Person_7,37,City_B,1719119186.90434,zeAZLtUoF/naig,Job_7





 the sum of ages of all people: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


Allowed shorthand string literals for **write_disposition**: 

append will always add new data at the end of the table. 

replace will replace existing data with new data. 

skip will prevent data from loading. 

"merge" will deduplicate and merge data based on "primary_key" and "merge_key" hints.

Defaults to "append".

In [26]:
# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(
    people_1(),
    table_name="people_new",
    write_disposition="replace",
)

info = generators_pipeline.run(
    people_2(),
    table_name="people_new",
    write_disposition="merge",
    primary_key="id"
)

print(info)

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_new table below:")

people_info = conn.sql("SELECT * FROM people_new").df()
display(people_info)


# calculate the sum of ages of all people
print(f"\n\n\n the sum of ages of all people: {people_info["age"].sum()}")

Pipeline dlt_ipykernel_launcher load step completed in 0.52 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\Nguyen\OneDrive\Documents\GitHub\data-engineering-zoomcamp\cohorts\2024\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1719119277.4259925 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_new          │
└─────────────────────┘




 people_new table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1719119276.5825698,Pod4k2DoVd45tA,None
1,2,Person_2,27,City_A,1719119276.5825698,2ZGt8u/8BL/J6w,None
2,7,Person_7,37,City_B,1719119277.4259925,GGEb9ZXzty8wIA,Job_7
3,5,Person_5,35,City_B,1719119277.4259925,wmcrNNyuOXfzUQ,Job_5
4,8,Person_8,38,City_B,1719119277.4259925,hubcX0TArpwNWQ,Job_8
5,3,Person_3,33,City_B,1719119277.4259925,sCLI1nLb4eM50A,Job_3
6,6,Person_6,36,City_B,1719119277.4259925,d12e3BZoD6eBzQ,Job_6
7,4,Person_4,34,City_B,1719119277.4259925,UgruK/xvpdicHA,Job_4





 the sum of ages of all people: 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX